# RFM ile Müşteri Segmentasyonu (Customer Segmentation with RFM)

1. İş Problemi (Business Problem)
2. Veriyi Anlama (Data Understanding)
3. Veri Hazırlama (Data Preparation)
4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)
6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)
7. Tüm Sürecin Fonksiyonlaştırılması

In [48]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [49]:
df_ = pd.read_csv("flo_data_20k.csv")
df = df_.copy()

In [50]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.000,1.000,139.990,799.380,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.000,1.000,49.990,159.990,[AKTIFSPOR]


In [51]:
df.columns

Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date',
       'last_order_date', 'last_order_date_online', 'last_order_date_offline',
       'order_num_total_ever_online', 'order_num_total_ever_offline',
       'customer_value_total_ever_offline', 'customer_value_total_ever_online',
       'interested_in_categories_12'],
      dtype='object')

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [53]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.000,3.111,4.226,1.000,1.000,2.000,4.000,200.000
order_num_total_ever_offline,19945.000,1.914,2.063,1.000,1.000,1.000,2.000,109.000
customer_value_total_ever_offline,19945.000,253.923,301.533,10.000,99.990,179.980,319.970,18119.140
customer_value_total_ever_online,19945.000,497.322,832.602,12.990,149.980,286.460,578.440,45220.130


In [54]:
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [55]:
df["TotalEver"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["TotalEverValue"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [56]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,TotalEver,TotalEverValue
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.000,1.000,139.990,799.380,[KADIN],5.000,939.370
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.000,2013.550
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]",5.000,585.320
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]",2.000,121.970
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.000,1.000,49.990,159.990,[AKTIFSPOR],2.000,209.980


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [58]:
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [60]:
# Alışveriş kanallarındaki müşteri sayısının, toplam alınan ürün sayısı ve toplam harcamaların dağılımı.
df.groupby("order_channel").agg({"master_id": "count",
                                 "TotalEver": "sum",
                                 "TotalEverValue": "sum"})     

,master_id,TotalEver,TotalEverValue
order_channel,,,
Android App,9495,52269.000,7819062.760
Desktop,2735,10920.000,1610321.460
Ios App,2833,15351.000,2525999.930
Mobile,4882,21679.000,3028183.160


In [61]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.000,3.111,4.226,1.000,1.000,2.000,4.000,200.000
order_num_total_ever_offline,19945.000,1.914,2.063,1.000,1.000,1.000,2.000,109.000
customer_value_total_ever_offline,19945.000,253.923,301.533,10.000,99.990,179.980,319.970,18119.140
customer_value_total_ever_online,19945.000,497.322,832.602,12.990,149.980,286.460,578.440,45220.130
TotalEver,19945.000,5.025,4.743,2.000,3.000,4.000,6.000,202.000
TotalEverValue,19945.000,751.244,895.402,44.980,339.980,545.270,897.780,45905.100


In [62]:
df.sort_values("TotalEverValue", ascending=False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,TotalEver,TotalEverValue
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.000,2.000,684.970,45220.130,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.000,45905.100
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.000,1.000,130.490,36687.800,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.000,36818.290
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.000,1.000,1263.760,32654.340,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.000,33918.100
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.000,1.000,538.940,30688.470,"[ERKEK, KADIN, AKTIFSPOR]",11.000,31227.410
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.000,3.000,18119.140,2587.200,[AKTIFSPOR],4.000,20706.340
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.000,4.000,843.680,17599.890,"[ERKEK, KADIN, AKTIFSPOR]",70.000,18443.570
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.000,1.000,82.480,16836.090,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.000,16918.570
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.000,1.000,180.730,12545.370,"[ERKEK, AKTIFSPOR]",37.000,12726.100
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.000,1.000,49.990,12232.250,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.000,12282.240
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,Desktop,Desktop,2018-11-24,2020-11-11,2020-11-11,2019-12-06,18.000,2.000,64.970,12038.180,"[ERKEK, KADIN]",20.000,12103.150


In [63]:
df.sort_values("TotalEver", ascending=False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,TotalEver,TotalEverValue
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.000,2.000,684.970,45220.130,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.000,45905.100
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.000,1.000,49.990,12232.250,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.000,12282.240
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.000,109.000,10239.460,143.980,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.000,10383.440
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.000,87.000,8432.250,139.980,[ERKEK],88.000,8572.230
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.000,81.000,3997.550,242.810,[ERKEK],83.000,4240.360
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.000,1.000,1263.760,32654.340,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.000,33918.100
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.000,74.000,4640.770,543.280,"[KADIN, AKTIFSPOR]",77.000,5184.050
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.000,3.000,292.930,5004.950,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.000,5297.880
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.000,1.000,82.480,16836.090,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.000,16918.570
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.000,4.000,843.680,17599.890,"[ERKEK, KADIN, AKTIFSPOR]",70.000,18443.570


In [64]:
def pre_proccesing(df):
    df.dropna(inplace=True)
    df["TotalEver"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
    df["TotalEverValue"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
    date_columns = df.columns[df.columns.str.contains("date")]
    df[date_columns] = df[date_columns].apply(pd.to_datetime)
    return df

In [65]:
df = pre_proccesing(df_)

In [66]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,TotalEver,TotalEverValue
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.000,1.000,139.990,799.380,[KADIN],5.000,939.370
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.000,2013.550
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]",5.000,585.320
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]",2.000,121.970
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.000,1.000,49.990,159.990,[AKTIFSPOR],2.000,209.980


RFM Metriklerinin Hesaplanması

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [68]:
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

In [69]:
today_date = dt.datetime(2021, 6, 1)

In [70]:
rfm = df.groupby("master_id").agg({"last_order_date": lambda date: (today_date - date.max()).days,
                                    "TotalEver": lambda num: num.sum(),
                                    "TotalEverValue": lambda price: price.sum()})

In [71]:
rfm.head()

,last_order_date,TotalEver,TotalEverValue
master_id,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.000,776.070
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.000,269.470
000be838-85df-11ea-a90b-000d3a38a36f,213,4.000,722.690
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.000,874.160
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.000,1620.330


In [72]:
rfm.columns = ["recency", "frequency", "monetary"]

In [73]:
rfm.head()

,recency,frequency,monetary
master_id,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.000,776.070
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.000,269.470
000be838-85df-11ea-a90b-000d3a38a36f,213,4.000,722.690
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.000,874.160
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.000,1620.330


In [74]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,19945.000,134.458,103.281,2.000,43.000,111.000,202.000,367.000
frequency,19945.000,5.025,4.743,2.000,3.000,4.000,6.000,202.000
monetary,19945.000,751.244,895.402,44.980,339.980,545.270,897.780,45905.100


In [75]:
rfm = rfm[(rfm["monetary"]) > 0 & (rfm["frequency"] > 0)]

In [76]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels = [5,4,3,2,1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels = [1,2,3,4,5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels = [1,2,3,4,5])

In [77]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score
master_id,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.000,776.070,5,4,4
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.000,269.470,1,2,1
000be838-85df-11ea-a90b-000d3a38a36f,213,4.000,722.690,2,3,4
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.000,874.160,5,4,4
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.000,1620.330,5,4,5


In [78]:
rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str) + rfm["monetary_score"].astype(str))
rfm["RF_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

In [79]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,RF_SCORE
master_id,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.000,776.070,5,4,4,544,54
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.000,269.470,1,2,1,121,12
000be838-85df-11ea-a90b-000d3a38a36f,213,4.000,722.690,2,3,4,234,23
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.000,874.160,5,4,4,544,54
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.000,1620.330,5,4,5,545,54


RFM Segmentlerinin Yapılması

In [80]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [81]:
rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)

In [82]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,RF_SCORE,segment
master_id,,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.000,776.070,5,4,4,544,54,champions
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.000,269.470,1,2,1,121,12,hibernating
000be838-85df-11ea-a90b-000d3a38a36f,213,4.000,722.690,2,3,4,234,23,at_Risk
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.000,874.160,5,4,4,544,54,champions
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.000,1620.330,5,4,5,545,54,champions


In [83]:
rfm.reset_index(inplace=True)

In [84]:
rfm.head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,RF_SCORE,segment
0,00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.000,776.070,5,4,4,544,54,champions
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.000,269.470,1,2,1,121,12,hibernating
2,000be838-85df-11ea-a90b-000d3a38a36f,213,4.000,722.690,2,3,4,234,23,at_Risk
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.000,874.160,5,4,4,544,54,champions
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.000,1620.330,5,4,5,545,54,champions


In [85]:
rfm[["segment","recency","frequency","monetary"]].groupby("segment").agg(["mean","count"])

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep      114.032  1643     2.407  1643  361.649  1643
at_Risk             242.329  3152     4.470  3152  648.325  3152
cant_loose          235.159  1194    10.717  1194 1481.652  1194
champions            17.142  1920     8.965  1920 1410.709  1920
hibernating         247.426  3589     2.391  3589  362.583  3589
loyal_customers      82.558  3375     8.356  3375 1216.257  3375
need_attention      113.037   806     3.739   806  553.437   806
new_customers        17.976   673     2.000   673  344.049   673
potential_loyalists  36.870  2925     3.311  2925  533.741  2925
promising            58.695   668     2.000   668  334.153   668

In [86]:
#Merge with interested category
df_interested = df[["master_id","interested_in_categories_12"]]
rfm = pd.merge(rfm, df_interested, on="master_id", how="left")

In [87]:
rfm.head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,RF_SCORE,segment,interested_in_categories_12
0,00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.000,776.070,5,4,4,544,54,champions,"[ERKEK, COCUK, AKTIFSPOR]"
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.000,269.470,1,2,1,121,12,hibernating,"[ERKEK, KADIN]"
2,000be838-85df-11ea-a90b-000d3a38a36f,213,4.000,722.690,2,3,4,234,23,at_Risk,"[AKTIFCOCUK, AKTIFSPOR]"
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.000,874.160,5,4,4,544,54,champions,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]"
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.000,1620.330,5,4,5,545,54,champions,"[ERKEK, AKTIFSPOR]"


In [88]:
cust_id = rfm[(rfm["segment"].isin(["champions","loyal_customers"])) & (rfm["interested_in_categories_12"].str.contains("KADIN"))]["master_id"]

In [89]:
cust_id.to_csv("yeni_urun_hedef_musteri_id.csv",index=False)

In [90]:
# Geçmişte iyi ama şimdi fazla alışveriş yapmayan müşteriler
sale_customers = rfm[(rfm["segment"].isin(["cant_loose","hibernating","at_Risk","new_customers","about_to_sleep"])) &((rfm["interested_in_categories_12"].str.contains("ERKEK"))|(rfm["interested_in_categories_12"].str.contains("COCUK")))]["master_id"]

In [91]:
sale_customers.to_csv("indirim_hedef_musteri_id.csv",index=False)

Çalışmanın Scripti

In [92]:
def create_rfm(df):
    df.dropna(inplace=True)
    df["TotalEver"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
    df["TotalEverValue"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
    date_columns = df.columns[df.columns.str.contains("date")]
    df[date_columns] = df[date_columns].apply(pd.to_datetime)

    today_date = dt.datetime(2021, 6, 1)
    rfm = df.groupby("master_id").agg({"last_order_date": lambda date: (today_date - date.max()).days,
                                    "TotalEver": lambda num: num.sum(),
                                    "TotalEverValue": lambda price: price.sum()})
    rfm.columns = ["recency", "frequency", "monetary"]
    rfm = rfm[(rfm["monetary"]) > 0 & (rfm["frequency"] > 0)]

    rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels = [5,4,3,2,1])
    rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels = [1,2,3,4,5])
    rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels = [1,2,3,4,5])

    rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str) + rfm["monetary_score"].astype(str))
    rfm["RF_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

    seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
    }

    rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)
    rfm.reset_index(inplace=True)

    df_interested = df[["master_id","interested_in_categories_12"]]
    rfm = pd.merge(rfm, df_interested, on="master_id", how="left")

    return rfm

In [93]:
new_rfm = create_rfm(df_)

In [94]:
new_rfm

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,RF_SCORE,segment,interested_in_categories_12
0,00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.000,776.070,5,4,4,544,54,champions,"[ERKEK, COCUK, AKTIFSPOR]"
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.000,269.470,1,2,1,121,12,hibernating,"[ERKEK, KADIN]"
2,000be838-85df-11ea-a90b-000d3a38a36f,213,4.000,722.690,2,3,4,234,23,at_Risk,"[AKTIFCOCUK, AKTIFSPOR]"
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.000,874.160,5,4,4,544,54,champions,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]"
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.000,1620.330,5,4,5,545,54,champions,"[ERKEK, AKTIFSPOR]"
...,...,...,...,...,...,...,...,...,...,...,...
19940,fff1db94-afd9-11ea-b736-000d3a38a36f,183,5.000,796.870,2,4,4,244,24,at_Risk,[AKTIFSPOR]
19941,fff4736a-60a4-11ea-8dd8-000d3a38a36f,67,2.000,318.870,4,2,2,422,42,potential_loyalists,[]
19942,fffacd34-ae14-11e9-a2fc-000d3a38a36f,257,6.000,983.920,1,4,4,144,14,at_Risk,[ERKEK]
19943,fffacecc-ddc3-11e9-a848-000d3a38a36f,132,9.000,1362.200,3,5,5,355,35,loyal_customers,"[COCUK, KADIN, AKTIFSPOR]"
